In [2]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
import time
from ctapipe.image import number_of_island_sizes
from ctapipe.image import (
    hillas_parameters,
    leakage_parameters,
    concentration_parameters,
)
from ctapipe.image import timing_parameters
from ctapipe.image import number_of_islands
from ctapipe.image import camera_to_shower_coordinates
import ctapipe
from lstchain.io import replace_config, standard_config
import h5py
from astropy.io import fits
import json
import os
from pyirf import spectral,simulations,utils 

## Observations

In [20]:
# Треба завантажити конфіг в івент соурс
filename = f'/Users/vdk/RealLST/LST-1.1.Run15085.0001.fits.fz'

int_cut = 0.3
min_pixel = 30

config_filename = '/Users/vdk/RealLST/typical_run_data/lstchain_standard_v0.10_heuristic_ff.json'
with open(config_filename) as json_file:
    config_data = json.load(json_file)

custom_config = Config(config_data)

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

cfg = Config({
    "source_config": {
        "EventSource" : {
            "allowed_tels": [1],"max_events": 9999}},
        "PointingSource":{
            "drive_report_path": '/Users/vdk/RealLST/data/DrivePosition_log_20231007.txt'},
        "LSTR0Corrections": {
          "calib_scale_high_gain":1.088,
          "calib_scale_low_gain":1.004,
          "drs4_pedestal_path": '/Users/vdk/RealLST/data/drs4_pedestal.Run14937.0000.h5',
          "calibration_path": '/Users/vdk/RealLST/data/calibration_filters_52.Run14938.0000.h5',
          "drs4_time_calibration_path": '/Users/vdk/RealLST/data/time_calibration.Run08349.0000.h5'
      }})

config = replace_config(standard_config, muon_processor_config)
config = replace_config(config, muon_processor_config_2)
config = replace_config(config, custom_config)


In [21]:
source = EventSource(filename, config = Config(cfg))
event_iterator = iter(source)

In [22]:
time_2 = []
for event in event_iterator:
    time_2.append(event.trigger.time.to_value(format = 'unix_tai'))

Calculating time reference values not from first event. This might result in wrong timestamps due to UCTS jumps
Using event 53003 as time reference for dragon. timestamp: 1697084503430796050 from ucts counter: 8430795800


ValueError: A value (1697084466.4307961) in x_new is above the interpolation range's maximum value (1696766701).

In [23]:
len(time_2)

0

In [24]:
time_2[-1] -  time_2[0]

IndexError: list index out of range

In [25]:
time_2[-1]

IndexError: list index out of range

In [34]:
time[-1]

1696720169.7323139

In [21]:
event.trigger.time.to_value(format = 'unix_tai')

1696720155.4783056

In [11]:
1696720155.4783056 - 1696720155.477298

0.0010075569152832031

## Simulations

In [131]:
muons_energy = [1.203954219818115234e+00, 1.407119393348693848e+00, 3.442558348178863525e-01, 7.431100308895111084e-02, 5.539828166365623474e-02, 6.081026196479797363e-01, 1.953697204589843750e+01, 1.592359095811843872e-01]

w_muons = sum([energy**(-0.7) for energy in muons_energy])

with open('/Users/vdk/runs31-41_protons.txt', 'r') as fi:
    lines1 = fi.readlines()

with open('/Users/vdk/runs41-51_protons.txt', 'r') as fi:
    lines2 = fi.readlines()

splitted = lines1[0].split(' ')
protons_energy1 = [float(i) for i in splitted]
splitted = lines2[0].split(' ')
protons_energy2 = [float(i) for i in splitted]

protons_energy = protons_energy1 + protons_energy2

w_protons = sum([energy**(-0.7) for energy in protons_energy])
print("Weighted muons = ", w_muons)
print("Weighted protons = ", w_protons)

Weighted muons =  22.68288225410579
Weighted protons =  13268.778060739964


In [7]:
Nshow = 20e4 * 10 * 500
E_min = 10
E_max = 103901
viewcone = 9
side = 1511.53
area = side**2
solid_angle = utils.cone_solid_angle(viewcone*u.deg).to_value()

# E^-2.7 from 10 to 10e5 GeV
energy_integral_true_spec = 0.01173 
# E^-2.7 from 10 to 103901 GeV
energy_integral_true_spec = 0.01172

# E^-0.7 from 10 to 10e5 GeV
energy_integral_weigthed_spec = 98.75 
# E^-0.7 from 10 to 103901 GeV
energy_integral_weigthed_spec = 99.97 

energy_integral_true_spec = 0.1


In [6]:
Nshow

1000000000.0

In [8]:
def intensity(area, solid_angle, energy_integral):
    N_0 = 1.8e4
    return N_0 * area * solid_angle * 0.74 * energy_integral

In [12]:
proton_rate = intensity(area=area, solid_angle = solid_angle, energy_integral=energy_integral_true_spec)
proton_number = Nshow * energy_integral_weigthed_spec
sim_time = proton_number / proton_rate
print(f"Simulated time {sim_time} sec")

Simulated time 424.6542392931994 sec


In [166]:
w_protons/sim_time

225.4501173689886

In [168]:
(w_muons*6)/sim_time

2.312432287153613

In [148]:
w_muons

22.68288225410579

In [45]:
17.96/3719

0.004829255176122614

In [98]:
4.771929824561403/3719

0.0012831217597637546

In [128]:

w_muons

22.68288225410579

13268.778060739964

In [174]:
w_muons/elapsed_time

15.878017577874054

In [175]:
166/elapsed_time

116.2

In [168]:
266/elapsed_time

62.51

In [169]:
116/9288

0.012489233419465978

In [100]:
166/1.42

116.90140845070422

In [120]:
protons_energy[0]**(-0.7)

0.05477001648814644

In [194]:
Nshow / intensity

1.0910441860127758

In [200]:
def intensity(area, solid_angle):
    return 1.8 * 10000 * solid_angle * area * 0.74

In [203]:
test = intensity(area, solid_angle)

In [204]:
Nshow / (test * integral)

1.4743840351523994

In [205]:
472/1.47

321.08843537414964

In [212]:
pyirf.__dir__()

['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__path__',
 '__file__',
 '__cached__',
 '__builtins__',
 '_version',
 'version',
 '__version__',
 '__all__',
 'binning',
 'cuts',
 'simulations',
 'io',
 'exceptions',
 'utils',
 'interpolation']

In [9]:
spectral.PDG_ALL_PARTICLE

PowerLaw(18000.0 1 / (GeV s sr m2) * (E / 1.0 GeV)**-2.7)

In [46]:
power_law = spectral.PowerLaw(normalization=1 * spectral.DIFFUSE_FLUX_UNIT,
        index=-2)

In [69]:
simulated_events = simulations.SimulatedEventsInfo(n_showers=int(20e6),
                                                   energy_min=10 * u.GeV,
                                                   energy_max=100 * u.TeV,
                                                   max_impact=1500 * u.m,
                                                   spectral_index=-2,
                                                   viewcone=10 * u.deg)

energy_bins = np.geomspace(simulated_events.energy_min, simulated_events.energy_max, 300)

In [75]:
#plt.hist(simulated_events.calculate_n_showers_per_energy(energy_bins), log = True)

sum(simulated_events.calculate_n_showers_per_energy(energy_bins))

20000000.00000001

In [53]:
power_law.from_simulation(simulated_events, 5*u.second)

PowerLaw(0.0592883209766677 1 / (TeV s sr m2) * (E / 1.0 TeV)**-2)

In [54]:
power_law.integrate_cone(0*u.deg, 10*u.deg)

PowerLaw(0.0954557030567379 1 / (TeV s m2) * (E / 1.0 TeV)**-2)

In [62]:
spectral.calculate_event_weights(100*u.TeV, spectral.PDG_ALL_PARTICLE, power_law)

0.00569209978830307

In [66]:
utils.cone_solid_angle(10*u.deg)

<Quantity 0.0954557 sr>

In [125]:
power_law = spectral.PowerLaw(normalization=1 * spectral.DIFFUSE_FLUX_UNIT,index=-2, e_ref= 1*  u.TeV)

w_protons = sum([spectral.calculate_event_weights(energy*u.TeV, spectral.PDG_ALL_PARTICLE, power_law) for energy in muons_energy])

In [126]:
w_protons

3.2431776874852627

In [118]:
help(spectral.PowerLaw)

Help on class PowerLaw in module pyirf.spectral:

class PowerLaw(builtins.object)
 |  PowerLaw(normalization, index, e_ref=<Quantity 1. TeV>)
 |  
 |  A power law with normalization, reference energy and index.
 |  Index includes the sign:
 |  
 |  .. math::
 |  
 |      \Phi(E, \Phi_0, \gamma, E_\text{ref}) =
 |      \Phi_0 \left(\frac{E}{E_\text{ref}}\right)^{\gamma}
 |  
 |  Attributes
 |  ----------
 |  normalization: astropy.units.Quantity[flux]
 |      :math:`\Phi_0`,
 |  index: float
 |      :math:`\gamma`
 |  e_ref: astropy.units.Quantity[energy]
 |      :math:`E_\text{ref}`
 |  
 |  Methods defined here:
 |  
 |  __call__(self, energy)
 |      Call self as a function.
 |  
 |  __init__(self, normalization, index, e_ref=<Quantity 1. TeV>)
 |      Create a new PowerLaw spectrum
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  integrate_cone(self, inner, outer)
 |      Integrate this powerlaw over solid angle in the given cone
 |      
 |      Parameters
 |      ------